# Описание проекта

Нужно разработать метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы.
Защитить данные пользователей, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

# Описание данных
- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [1]:
# Импорт стандартной библиотеки
import pandas  as pd
import numpy as np
# Сторонний импорт
from numpy import linalg
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression


In [2]:
data.info()
display(data.head())
print()
display(data.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


**Значения без аномалий. Пропусков нет. Возраст можно преобразовать в целое число, а зарплаты округлить.**

## Умножение матриц

- X - матрица признаков (наш исходный датасет)
- w - вектор весов линейной регрессии
- a - вектор предсказаний линейной регессии
- y - вектор значений целевого признака
- P - случайно сгенерированная квадратная матрица, длина и ширина которой равны количеству признаков (в нашем случае 4х4), эта матрица должна быть обратимой, то есть должна существовать такая матрица $P^{-1}$, чтобы $PP^{-1}=E$

Известно, что для линейной регрессии верны следующие выражения:

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Задача:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

**Ответ:** не изменится.

Заменим X матрицу на закодированную XP и вычислим, чему будет равно предсказание и вектор весов.

$$
a' = (XP)w'
$$ 

$$
w' = (XP^T XP)^{-1} XP^T y
$$

$$
a' = XP((XP)^T (XP))^{-1} (XP)^T y 
$$

Далее нам потребуются следующие правила обратной матрицы, что бы раскрыть скобки и найти связь между w и w':

1. Одно из свойств транспонированных матриц: Транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке.

$$
(AB)^T = B^T A^T 
$$

2. Далее помним, что нужно соблюдать порядок множителей, так как умножение матриц в общем случае некоммутативно:
$$
AB != BA
$$
    
3. Обратная матрица от произведения двух матриц равна произведению обратных матриц взятых в обратном порядке. 
$$
(AB)^{-1} = B^{-1} A^{-1} 
$$
Для трех матриц это выглядит так: 
$$
(ABC)^{-1} = C^{-1} B^{-1} A^{-1} 
$$

Это свойство действительно только для квадратных матриц. В нашем случае квадратные матрицы $P^T$, $(X^TX)$ и $P$. 

Раскроем скобки: 

$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

`PP^{-1} = (P^T)^{-1} P^T =  E` - едничная матрица. Умножение на единичную матрицу результат не меняет.
$$
PP^{-1} = (P^T)^{-1} P^T =  E
$$

Следовательно, 

$$
w' = P^{-1} (X^T X)^{-1} X^T y
$$

$$
a' = (XP) P^{-1} (X^T X)^{-1} X^T y = XE (X^T X)^{-1} X^T y
$$ 

$$
XE = EX = X
$$ 

$$
a = X (X^T X)^{-1} X^T y = XE (X^T X)^{-1} X^T y
$$

$$
a' = a
$$

Доказали, что при умножении матрицы `(X или XP)` признаков на обратную матрицу, значение `a = a'`. Значит `a`  не меняется.

## Алгоритм преобразования

Чтобы выполнить проверку алгоритма нужно

1. обучить линейную регрессию на исходных признаках

2. найти R2

3. создать случайную матрицу

4. проверить ее на обтатимость

5. закодировать наши признаки умножив их на кодирующую матрицу

6. обучить линейную регрессию на закодированных призанаках, рассчитать R2

7. сравнить R2 до кодирования и после

## Проверка алгоритма.

In [3]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

# обучим линейную регрессию на исходных признаках
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)

# Найдем R2
score_before = r2_score(target, predictions)

# Сгенерируем случайную матрицу функцией
𝑃 = np.random.randint(100, size = (4, 4))        
P_inv = np.linalg.inv(P) 

# Проверим правильность результата, опираясь на основное свойство обратной матрицы, т.е. np.dot(a, ainv) = np.dot(ainv, a) = np.eye(a.shape[0])
print('\nСвойство обратной матрицы. Результат верный: ', np.allclose(np.dot(𝑃, 𝑃_inv), np.eye(𝑃.shape[0])),
                                                       np.allclose(np.dot(𝑃_inv, 𝑃), np.eye(𝑃.shape[0])))

# Закодируем наши признаки умножив их на кодирующую матрицу
features = features.dot(P)

# Обучим линейную регрессию на закодированных призанаках, рассчитаем R2
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
score_after = r2_score(target, predictions)
 
# Сравним результаты
print(f'Сравним R2 до {score_before} и после {score_after}')



Свойство обратной матрицы. Результат верный:  True True
Сравним R2 до 0.4249455028666801 и после 0.42494550286668


## Общий вывод: 
- Загрузили и  изучили данные. Значения без аномалий. Пропусков нет.
- Доказали матеметически, что при умножении признаков как случайной, так и исходной матрицы, на обратимую матрицу, качество линейной регрессии не изменится. Т.е её можно обучить заново. Для этого обратились к правилам обратной, транспонированной и единичной матриц. Раскрыли скобки уровнения вектора весов, сократили что возможно. Нашли связь между `w и w'`, доказали равенство `a=a'`.
- Создали алгоритм преобразования данных X в XP. Проверили свойство обратимости для случайно сгенерированной матрицы. Закодировали признаки умножив их на кодирующую матрицу. Обучили линейную регрессию на закодированных призанаках и  рассчитали R2 до и после.
- Результаты счета R2 до и после кодирования практически полностью совпадают. 